# Day 3: Learning Structured Predictors

In Day 2, we focused on generative sequence classifiers - HMMs. Today's focus is on discriminative classifiers. Recall that:

* **generative classifiers** try to model the probability distribution of the data, $P(X, Y)$;

* **discriminative classifiers** only model the conditional probability of each class given the observed data, $P(Y\,|\,X)$.

In Day 1, we implemented discriminative models for classification tasks. Today, we extend this concept to the classification of _sequential_ data.

## Summary

You will be using two discriminative classifiers to do part-of-speech tagging:
* Conditional Random Fields (CRF) and
* Structured Perceptron.

Your tasks for this lab session are:

* to train a CRF model using two different sets of features (exercises 3.1 and 3.2); 
* to implement the structured perceptron algorithm (exercise 3.3); 
* to compare the performance of the Structured Perceptron with that of CRFs (exercise 3.4).

## Introduction

The discriminative models covered in Day 1 search the solution of

[BLAH eq for normal classification models]

Similarly, discriminative sequence models aim to solve the following:

$$\underset{y\,\in\,\Lambda^N}{\textrm{arg max}}\ P(Y=y\,|\,X=x)=\underset{y\,\in\,\Lambda^N}{\textrm{arg max}}\ \boldsymbol{w}\cdot\boldsymbol{f}(x, y)$$

where $\boldsymbol{w}$ is the model's weight vector, and $\boldsymbol{f}(x, y)$ is a feature vector.

In Day 2, sequences were scored using the log-probability. On today's models we are still scoring the sequences; the only difference is the scores are now computed as the product of the weights with the feature vector:


| score | Hidden Markov Models (Day 2) | Discriminative Models (Today) |
| ------------------------------- | ---------------- | ---------------- |
| $\textrm{score}_\textrm{emiss}$ | $\log P(x_i\,|\,y_i) $ | $\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{emiss}(i, x, y_i)$ |
| $\textrm{score}_\textrm{init}$ | $\log P(y_1\,|\,\mathrm{start}) $ | $\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{init}(x, y_1)$ |
| $\textrm{score}_\textrm{trans}$ | $\log P(y_{i+1}\,|\,y_i) $ | $\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{trans}(i, x, y_i, y_{i+1})$ |
| $\textrm{score}_\textrm{final}$ | $\log P(\mathrm{stop}\,|\,y_N) $ | $\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{final}(x, y_N)$ |

Notice that the scores computed using the feature vector depend on two sequential values of the output variable, $y$, but may depend on the whole observated input, $x$. We can now rewrite the above expression as

$$
\underset{y\,\in\,\Lambda^N}{\textrm{arg max}}\ 
\sum_{i=1}^N \boldsymbol{w}\cdot\boldsymbol{f}_\textrm{emiss}(i, x, y_i) + 
\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{init}(x, y_1) + 
\sum_{i=1}^{N-1}\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{trans}(i, x, y_i, y_{i+1}) + 
\boldsymbol{w}\cdot\boldsymbol{f}_\textrm{final}(x, y_N) = 
\\
\underset{y\,\in\,\Lambda^N}{\textrm{arg max}}\ 
\sum_{i=1}^N \textrm{score}_\textrm{emiss}(i, x, y_i) + 
\textrm{score}_\textrm{init}(x, y_1) + 
\sum_{i=1}^{N-1}\textrm{score}_\textrm{trans}(i, x, y_i, y_{i+1}) +
\textrm{score}_\textrm{final}(x, y_N)$$

The advantage of these linear models over HMMs is that the feature vector may contain more general features. [BLAH explain this]

One important thing to notice is that the decoding process will be unchanged. This means *we do not need to develop new decoders.* We will be using the Viterbi and Forward-Backward algorithms developed on the previous day.

## Notation

## Conditional Random Fields

CRFs are the generalization of the Maximum Entropy classifier for sequences.

Objectives:

* train a CRF using different feature sets for part-of-speech tagging;
* evaluate the model on the training, development and test sets.

Files used:

* class CRFOnline in lxmls/sequences/crf_online.py file
* class PostagCorpus in lxmls/sequences/readers/pos_corpus.py file
* class IDFeatures in lxmls/sequences/id_feature.py file
* class ExtendedFeatures in lxmls/sequences/extended_feature.py file

### Exercise 3.1 - Basic feature set

Start by training the model:

In [ ]:
import lxmls.sequences.crf_online as crfo
import lxmls.readers.pos_corpus as pcc
import lxmls.sequences.id_feature as idfc
import lxmls.sequences.extended_feature as exfc

print "CRF Exercise"

# Load the corpus
corpus = pcc.PostagCorpus()

# Load the training, test and development sequences
train_seq = corpus.read_sequence_list_conll("data/train-02-21.conll", max_sent_len=10, max_nr_sent=1000)
test_seq = corpus.read_sequence_list_conll("data/test-23.conll", max_sent_len=10, max_nr_sent=1000)
dev_seq = corpus.read_sequence_list_conll("data/dev-22.conll", max_sent_len=10, max_nr_sent=1000)

# Build features
feature_mapper = idfc.IDFeatures(train_seq)
feature_mapper.build_features()

# Train the model
# You will receive feedback when each epoch is finished.
# Note that running the 20 epochs might take a while.
crf_online = crfo.CRFOnline(corpus.word_dict, corpus.tag_dict, feature_mapper)
crf_online.num_epochs = 20
crf_online.train_supervised(train_seq)

Next, evaluate the model on the various datasets:

In [ ]:
# Make predictions for the various sequences using the trained model.
pred_train = crf_online.viterbi_decode_corpus(train_seq)
pred_dev = crf_online.viterbi_decode_corpus(dev_seq)
pred_test = crf_online.viterbi_decode_corpus(test_seq)

# Evaluate and print accuracies
eval_train = crf_online.evaluate_corpus(train_seq, pred_train)
eval_dev = crf_online.evaluate_corpus(dev_seq, pred_dev)
eval_test = crf_online.evaluate_corpus(test_seq, pred_test)
print "CRF - ID Features Accuracy Train: %.3f Dev: %.3f Test: %.3f"%(eval_train,eval_dev, eval_test)

Your output should be similar to this:

_CRF - ID Features Accuracy Train: 0.949 Dev: 0.846 Test: 0.858_

Compare with the results achieved with the HMM model (0.837 on the test set). Even when using a similar feature set, a CRF yields better results than the HMM from the previous lecture.

Perform some error analysis and figure out what are the main errors the tagger is making. Compare them with the errors made by the HMM model. 

**Hint:** use the methods developed in the previous lecture to help you with the error analysis.

### Exercise 3.2 - Extended feature set

Train the model again, this time using the extended feature set:

In [ ]:
# Build features
feature_mapper = exfc.ExtendedFeatures(train_seq)
feature_mapper.build_features()

# Train the model
# You will receive feedback when each epoch is finished.
# Note that running the 20 epochs might take a while.
crf_online = crfo.CRFOnline(corpus.word_dict, corpus.tag_dict, feature_mapper)
crf_online.num_epochs = 20
crf_online.train_supervised(train_seq)

And compute its accuracy:

In [ ]:
# Make predictions for the various sequences using the trained model.
pred_train = crf_online.viterbi_decode_corpus(train_seq)
pred_dev = crf_online.viterbi_decode_corpus(dev_seq)
pred_test = crf_online.viterbi_decode_corpus(test_seq)

# Evaluate and print accuracies
eval_train = crf_online.evaluate_corpus(train_seq, pred_train)
eval_dev = crf_online.evaluate_corpus(dev_seq, pred_dev)
eval_test = crf_online.evaluate_corpus(test_seq, pred_test)
print "CRF - ID Features Accuracy Train: %.3f Dev: %.3f Test: %.3f"%(eval_train,eval_dev, eval_test)

Your output should be similar to this:

_CRF - Extended Features Accuracy Train: 0.984 Dev: 0.899 Test: 0.894_

Compare the errors obtained with the two different feature sets. Do some error analysis: what errors were correct by using more features? Can you think of other features to use to solve the errors you found?

## Structured Perceptron

### Exercise 3.3 - Algorithm implementation

### Exercise 3.4 - POS Tagging